# Comparison to Alternative Methods

Explanation

In [1]:
import numpy as np
from tqdm.notebook import tqdm
np.random.seed(0)

def create_fake_data(n_timepoints_train, n_timepoints_test, n_features, n_targets, noise_amounts):
    # Create features
    X_train = np.random.rand(n_timepoints_train, n_features)
    X_test = np.random.rand(n_timepoints_test, n_features)
    true_weights = np.random.randn(n_features, n_targets)

    # Create targets
    Y_train = X_train @ true_weights
    Y_test = X_test @ true_weights

    # Add different amounts of noise
    for target_i in range(n_targets):
        Y_train[:,target_i] += noise_amounts[target_i]  * np.random.randn(n_timepoints_train)
        Y_test[:,target_i] += noise_amounts[target_i] * np.random.randn(n_timepoints_test)
    
    return X_train, X_test, Y_train, Y_test


n_timepoints_train, n_timepoints_test = 500, 100
n_features, n_targets = 10, 3
noise_amounts = [1,5,15]

X_train, X_test, Y_train, Y_test = create_fake_data(
    n_timepoints_train=n_timepoints_train,
    n_timepoints_test=n_timepoints_test,
    n_features=n_features,
    n_targets=n_targets,
    noise_amounts=noise_amounts,
    )

As in the example notebook, we'll compute our main regression fit using Himalaya.

In [2]:
from himalaya.ridge import RidgeCV

model = RidgeCV(alphas=np.logspace(-2, 5, 8))
model.fit(X_train, Y_train)
himalaya_scores = model.score(X_test, Y_test)
print("R2 Score Per Target: ", himalaya_scores)

R2 Score Per Target:  [ 0.50214792  0.0137505  -0.0056149 ]


First, we'll compute p value for R2 of each of our targets using COPTeRR.

In [3]:
from copterr import PermuteWeights

permuter = PermuteWeights(X_train, Y_train, model.best_alphas_)
permuter.prepare()

himalaya_weights = model.coef_
copterr_weights = permuter.fit_true_weights()
print('Weights Equivalent:', np.allclose(himalaya_weights, copterr_weights, atol=1e-5))

copterr_scores = permuter.score(X_test, Y_test).numpy()
print('R2 Scores Equivalent:', np.allclose(himalaya_scores, copterr_scores, atol=1e-5))

Computing Initial SVDs: 100%|██████████| 3/3 [00:00<00:00, 2600.85it/s]

Weights Equivalent: True
R2 Scores Equivalent: True


Compute with copterr

In [4]:
from copterr.utils import compute_p_values

perm_performance = []
for permutation in tqdm(range(10000)):
    perm_weights = permuter.fit_permutation(permutation=True)
    perm_r2 = permuter.score(X_test, Y_test, permutation=True)
    perm_performance.append(perm_r2.numpy())

p_values = compute_p_values(copterr_scores, perm_performance)

for target_i in range(n_targets):
    print(f"Target {target_i} R2:", "%0.3f" % copterr_scores[target_i], "    ", "P-Value:", "%0.2f" % p_values[target_i])

  0%|          | 0/10000 [00:00<?, ?it/s]

Target 0 R2: 0.502      P-Value: 0.00
Target 1 R2: 0.014      P-Value: 0.01
Target 2 R2: -0.006      P-Value: 0.11


A common way to estimate significance for regression models is by shuffling the input features, and then fully re-estimating the regression weights. We'll test this by repeatedly shuffling and fitting using Himalaya.

In [5]:
from himalaya.ridge import Ridge
from copterr.utils import compute_p_values

permutation_model = Ridge(alpha=model.best_alphas_)
X_train_shuffled = X_train.copy()
X_test_shuffled = X_test.copy()

perm_performance = []
for i in tqdm(range(10000)):
    np.random.shuffle(X_train_shuffled)
    np.random.shuffle(X_test_shuffled)
    permutation_model.fit(X_train_shuffled, Y_train)
    perm_r2 = permutation_model.score(X_test_shuffled, Y_test)
    perm_performance.append(perm_r2)

  0%|          | 0/10000 [00:00<?, ?it/s]

In [6]:
p_values = compute_p_values(himalaya_scores, perm_performance)

for target_i in range(n_targets):
    print(f"Target {target_i} R2:", "%0.3f" % himalaya_scores[target_i], "    ", "P-Value:", "%0.2f" % p_values[target_i])

Target 0 R2: 0.502      P-Value: 0.00
Target 1 R2: 0.014      P-Value: 0.01
Target 2 R2: -0.006      P-Value: 0.11


Alternatively, we can do it the copterr way, but without copterr optimizations

In [7]:
from himalaya.ridge import Ridge
from copterr.utils import compute_p_values

permutation_model = Ridge(alpha=model.best_alphas_)
Y_train_shuffled = Y_train.copy()
Y_test_shuffled = Y_test.copy()

perm_performance = []
for i in tqdm(range(10000)):
    np.random.shuffle(Y_train_shuffled)
    np.random.shuffle(Y_test_shuffled)
    permutation_model.fit(X_train, Y_train_shuffled)
    perm_r2 = permutation_model.score(X_test, Y_test_shuffled)
    perm_performance.append(perm_r2)

  0%|          | 0/10000 [00:00<?, ?it/s]

In [8]:
p_values = compute_p_values(himalaya_scores, perm_performance)

for target_i in range(n_targets):
    print(f"Target {target_i} R2:", "%0.3f" % himalaya_scores[target_i], "    ", "P-Value:", "%0.2f" % p_values[target_i])

Target 0 R2: 0.502      P-Value: 0.00
Target 1 R2: 0.014      P-Value: 0.01
Target 2 R2: -0.006      P-Value: 0.11
